In [32]:
import logging
import os.path
import argparse
from typing import Any, Union

# External modules
import xarray as xr
import numpy as np
from tqdm import tqdm

# Internal modules

def get_regridder(
        grid_in: xr.Dataset, out_res: float, method='bilinear'
) -> Any:
    import xesmf as xe
    grid_out = xr.Dataset(
        {
            'lat': (['lat'], np.arange(-90+out_res/2, 90, out_res)),
            'lon': (['lon'], np.arange(0, 360, out_res)),
        }
    )
    regridder = xe.Regridder(grid_in, grid_out, method=method,
                             periodic=True, reuse_weights=False)
    return regridder


def _regrid_loop(
        regridder: Any,
        ds: xr.Dataset,
        chunk_size: Union[int, None] = None
) -> xr.Dataset:
    if chunk_size is None:
        chunk_size = len(ds['time'])
    ds_regridded = []
    regrid_pbar = tqdm(range(0, len(ds['time']), chunk_size))
    for chunk in regrid_pbar:
        tmp_regridded = regridder(
            ds.isel(time=slice(chunk, chunk+chunk_size))
        )
        ds_regridded.append(tmp_regridded)
    ds_regridded = xr.concat(ds_regridded, dim='time')
    for var in ds_regridded:
        ds_regridded[var].attrs = ds[var].attrs
    ds_regridded.attrs = ds.attrs
    return ds_regridded


def regrid_ds(
        ds_ens: Union[xr.Dataset, None],
        chunk_size: Union[int, None] = None
) -> Union[xr.Dataset, None]:
    if ds_ens is not None:
        ds_ens = ds_ens.rename({'latitude': 'lat', 'longitude': 'lon'})
        grid_in = ds_ens[['lat', 'lon']]
        regridder = get_regridder(grid_in, out_res=5.625)
        ds_regridded = _regrid_loop(regridder, ds_ens, chunk_size)
    else:
        ds_regridded = None
    return ds_regridded

In [6]:
import xesmf

In [27]:
y = xr.open_dataset(r"D:\Projects\NIOT\v1\ensemble_transformer\data\sfc_202312.nc")

In [28]:
y

<xarray.Dataset>
Dimensions:    (latitude: 1, longitude: 1, number: 10, time: 62)
Coordinates:
  * longitude  (longitude) float32 78.0
  * latitude   (latitude) float32 10.0
  * number     (number) int32 1 2 3 4 5 6 7 8 9 10
  * time       (time) datetime64[ns] 2023-12-03 ... 2024-01-02T12:00:00
Data variables:
    t2m        (time, number, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-06-13 05:16:05 GMT by grib_to_netcdf-2.33.2: /opt/ecmw...

In [33]:
z = regrid_ds(y, chunk_size=100)

ValueError: ESMC_FieldRegridStore failed with rc = 6. Please check the log files (named "*ESMF_LogFile").